In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"marcellopoliti","key":"810de577597fed0b321973b6c8894d21"}'}

In [2]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download "kingburrito666/shakespeare-plays"

100% 4.55M/4.55M [00:00<00:00, 45.9MB/s]



In [4]:
import zipfile

zip_ref = zipfile.ZipFile("/content/shakespeare-plays.zip")
zip_ref.extractall()

In [5]:
import pandas as pd

size = 10_000

df = pd.read_csv("/content/Shakespeare_data.csv")
df.head()
df = df[:size]

In [6]:
len(df)

10000

In [7]:
corpus = df.PlayerLine

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
oov_token = "<OOV>"
truncating_type = "pre"

In [10]:
tokenizer = Tokenizer(oov_token = oov_token)
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
vocab_size = len(word_index)

In [11]:
sequences = tokenizer.texts_to_sequences(corpus)

In [12]:
all_sequences = []

for seq in sequences:
  for i in range(len(seq)):
    n_gram = seq[0:i+1]
    all_sequences.append(n_gram)

In [13]:
max_len = max([len(seq) for seq in all_sequences])
max_len

56

In [14]:
padded = pad_sequences(all_sequences, maxlen = max_len, padding= "pre")

In [15]:
x = padded[:,:-1]
y = padded[:,-1]

In [16]:
from tensorflow.keras.layers import Embedding, Dense,LSTM,GRU,Bidirectional,Flatten,Conv1D
from tensorflow.keras.models import Sequential

##Let's import pre-trained embedding

In [17]:
!wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip ./glove.6B.100d.txt

--2022-02-14 08:26:34--  https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip
Resolving huggingface.co (huggingface.co)... 34.204.221.201, 34.224.55.150, 54.205.126.93, ...
Connecting to huggingface.co (huggingface.co)|34.204.221.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/stanfordnlp/glove/6471382cdd837544bf3ac72497a38715e845897d265b2b424b4761832009c837 [following]
--2022-02-14 08:26:34--  https://cdn-lfs.huggingface.co/stanfordnlp/glove/6471382cdd837544bf3ac72497a38715e845897d265b2b424b4761832009c837
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 13.226.9.86, 13.226.9.100, 13.226.9.123, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|13.226.9.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  41.6MB/s    in 19s     

2022-02-14 

In [18]:
zip_ref = zipfile.ZipFile("/content/glove.6B.zip")
zip_ref.extractall()

In [19]:
import numpy as np

embeddings_index = {};
embedding_dim = 100

with open('/content/glove.6B.100d.txt', encoding="utf-8",errors='ignore') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

In [20]:
model = Sequential([
  Embedding(input_dim=vocab_size + 1, output_dim=100, input_length=max_len - 1, weights = [embeddings_matrix], trainable = False),
  Conv1D(filters = 64, kernel_size = 5, activation = "relu"),
  Bidirectional(LSTM(64)),
  Dense(64,activation = "relu"),
  Dense(vocab_size + 1,"softmax")
])
  
model.summary()
model.compile(loss = "categorical_crossentropy", metrics = ["accuracy"], optimizer = "adam")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 55, 100)           785800    
                                                                 
 conv1d (Conv1D)             (None, 51, 64)            32064     
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 7858)              510770    
                                                                 
Total params: 1,402,938
Trainable params: 617,138
Non-trainable params: 785,800
__________________________________________

In [21]:
from tensorflow import keras
import tensorflow as tf

In [22]:
early_stopping = keras.callbacks.EarlyStopping(monitor = "loss",patience = 50)

In [23]:
y = tf.one_hot(y,vocab_size+1)

In [ ]:
history = model.fit(x,y,epochs = 500, callbacks = [early_stopping])

Epoch 1/500
2389/2389 [==============================] - 95s 34ms/step - loss: 6.8217 - accuracy: 0.0466
Epoch 2/500
2389/2389 [==============================] - 81s 34ms/step - loss: 6.3800 - accuracy: 0.0648
Epoch 3/500
2389/2389 [==============================] - 81s 34ms/step - loss: 6.1476 - accuracy: 0.0721
Epoch 4/500
2389/2389 [==============================] - 82s 34ms/step - loss: 5.9896 - accuracy: 0.0775
Epoch 5/500
2389/2389 [==============================] - 82s 34ms/step - loss: 5.8482 - accuracy: 0.0820
Epoch 6/500
2389/2389 [==============================] - 82s 34ms/step - loss: 5.7241 - accuracy: 0.0867
Epoch 7/500
2389/2389 [==============================] - 82s 34ms/step - loss: 5.6183 - accuracy: 0.0893
Epoch 8/500
2389/2389 [==============================] - 82s 34ms/step - loss: 5.5309 - accuracy: 0.0914
Epoch 9/500
2389/2389 [==============================] - 82s 34ms/step - loss: 5.4517 - accuracy: 0.0935
Epoch 10/500
2389/2389 [==============================]